<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/Manon_feature_selection/Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib

In [0]:
# General packages not sure anymore what I'm actually using lol
import numpy as np 
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import impute
from sklearn import datasets, linear_model
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn import feature_selection 
from sklearn import feature_selection 
from sklearn.model_selection import train_test_split

In [5]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] 
Y.replace(('AD', 'CN'), (1, 0), inplace=True) # convert AD and CN to 1 and 0

The number of samples: 855
The number of columns: 268


In [6]:
# split into train (70%) and test (30%) set

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(598, 267) (598,)
(257, 267) (257,)


In [7]:
# Preprocessing: drop feature if too many missing values

X_train = X_train.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_train.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_train)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_train.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_train.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_train.shape, y_train.shape)

102
(598, 261) (598,)


In [7]:
# Preprocessing:  impute missing values (fill missing)

imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_train_imp = imputer.fit_transform(X_train)          # impute  
X_train_imp = pd.DataFrame(data=X_train_imp, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

type(X_train_imp)


pandas.core.frame.DataFrame

In [8]:
# Preprocessing: scaling either standard or robust (removes median and scales data according to quantile range) 

scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_train_imp)
X_train_scaled = scaler.transform(X_train_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df
X_train_scaled


,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_021_S_0642_bl_0,-0.160235,1.656363,-1.461417,-0.194924,-1.173729,-0.518043,-1.644104,-0.251573,2.366362,1.698972,0.807054,2.361114,0.350970,-0.144765,-0.146918,0.507771,1.353871,1.092985,-0.678713,-0.936874,-1.002432,-0.814865,1.780225,2.087918,-0.040429,1.416965,1.118795,1.018903,1.397031,1.194431,0.589138,-0.078270,0.242154,-0.302968,-0.274393,-0.465838,0.106534,1.887010,1.561567,-0.753357,...,-0.253095,1.114315e+00,1.376069,-0.436705,3.767620,1.007494,-0.858368,2.481822,2.248571,1.448393,0.720203,0.426721,2.052465,2.486391,-0.305571,2.619735,3.767620,1.007494,-0.858368,2.481822,2.248571,1.448393,0.720203,0.426721,2.052465,2.486391,-0.305571,2.619735,4.757970e-01,1.090266,-0.331174,0.716762,8.366559e-01,8.653206e-01,-0.668074,0.262689,7.285633e-01,0.833764,-6.356775e-01,0.718112
0_135_S_4566_bl_0,-0.151509,-1.080483,0.650771,-0.598985,0.129008,-0.080460,0.919515,0.304470,-0.776296,-1.478649,-0.012291,-1.223840,1.135589,2.073184,0.166404,-1.444466,-2.212356,-1.020959,1.572426,2.474097,0.862103,0.473729,-0.377544,-0.520251,-0.029652,0.148016,-0.380564,0.632269,0.436452,-0.191677,-1.357495,-0.049057,-0.472530,1.247069,0.929014,1.089017,-1.069410,-0.508104,-1.038297,1.149759,...,-0.253094,-9.424061e-01,-0.566493,0.662380,-0.784234,-0.405931,0.634856,-1.007747,-1.177728,-1.143652,-0.718950,0.185360,-1.046934,-0.992557,0.653745,-1.034316,-0.784234,-0.405931,0.634856,-1.007747,-1.177728,-1.143652,-0.718950,0.185360,-1.046934,-0.992557,0.653745,-1.034316,-2.877971e-01,-0.072593,-0.354456,-1.072831,-1.088411e+00,-9.692346e-01,-0.667930,-0.438268,-1.027272e+00

In [0]:
# Code for feature selection
'''
First determine the best amount of features
'''

# Create the RFE object and compute a cross-validated score.
svc = svm.SVC(kernel="linear", random_state= None)
X= X_train_scaled
y= y_train

In [13]:


# classifications
rfecv = feature_selection.RFECV(
    estimator=svc, step=1, 
    cv=model_selection.StratifiedKFold(n_splits=5),
    scoring='roc_auc')
rfecv.fit(X, y)
feature_selection.RFECV()
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


KeyboardInterrupt: ignored

In [14]:
# Code for feature selection
'''
With the amount of features desired known, find the best performing features
'''
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
 
#apply SelectKBest class to extract top ... best features
bestfeatures = SelectKBest(score_func= f_classif, k=32) # k= number of desired features
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(32,'Score'))  #print ... best features 


                                            Specs       Score
1                                   (hf_entropy,)  540.044084
2                                  (hf_kurtosis,)  404.552131
11                                      (hf_std,)  364.426795
8                            (hf_quartile_range,)  353.618383
218                          (tf_LBP_std_R3_P12,)  349.047098
99   (tf_GLRLM_GrayLevelNonUniformityNormalized,)  282.934329
100                 (tf_GLRLM_GrayLevelVariance,)  281.213410
111                  (tf_GLRLM_ShortRunEmphasis,)  280.321212
222                        (tf_NGTDM_Complexity,)  279.807015
113      (tf_GLRLM_ShortRunLowGrayLevelEmphasis,)  257.356041
73              (phasef_phasesym_entropy_WL3_N5,)  238.773800
134                      (tf_Gabor_0.05A0.0skew,)  226.395155
6                                       (hf_min,)  207.733146
150                     (tf_Gabor_0.05A2.36mean,)  188.879981
9                                     (hf_range,)  187.038136
79      

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [208 211 212 213] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [8]:
# Code for PCA
from sklearn import decomposition
pca = decomposition.PCA(n_components=260)
X= X_train_scaled
test= X_test_scaled
pca.fit(X)
X_train_pca = pca.transform(X)
y= y_train
# X_pca= pd.DataFrame(data= X_train_pca, index=[X.index], columns=[X.columns])
X_test_pca = pca.transform(test)
print(X_train_pca)
#X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns])
seaborn.scatterplot(x=X_train_pca[:, 0], y=X_train_pca[:, 1], hue=y)

NameError: ignored

In [0]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');